In [40]:
#!/usr/bin/env python2
# encoding: utf-8

import os
import argparse
from tf.transformations import euler_from_quaternion, quaternion_from_euler
import pandas as pd
import matplotlib.pylab as plt
import re
import rosbag

### 初始化参数

In [63]:
  class Foo:
        
    def __init__(self):
        self.husky_topic = '/vrpn_client_node/husky/pose'
        self.mavebase_topic = '/move_base/result'
        self.rosbag_file_name = 'husky_nav_exp_2018-12-18-15-18-02.bag'
    
    def detail(self):
        print(self.husky_topic)
        print(self.mavebase_topic)
        print(self.rosbag_file_name)

In [62]:
foo = Foo()
foo.detail()
%run bag_to_pose_vrpn_angle.py husky_nav_exp_2018-12-18-15-18-02.bag /vrpn_client_node/husky/pose

/vrpn_client_node/husky/pose
/move_base/result
husky_nav_exp_2018-12-18-15-18-02.bag
Extract pose from bag husky_nav_exp_2018-12-18-15-18-02.bag in topic /vrpn_client_node/husky/pose
Saving to file /home/dzsb078/Desktop/201812121312_husky_nav/code_merge/husky_poses.csv
wrote 239583 husky pose messages to the file: /home/dzsb078/Desktop/201812121312_husky_nav/code_merge/husky_poses.csv


In [ ]:
self.husky_topic

### bag_to_pose_vrpn_angle.py

In [ ]:
def bag_to_pose_vrpn_angle(topic):
    
    roll = pitch = yaw = 0.0

    def extract(bagfile, pose_topic, msg_type, out_filename):
        n = 0
        f = open(out_filename, 'w')
        f.write('timestamp,x,y,z,roll,pitch,yaw\n')
        with rosbag.Bag(bagfile, 'r') as bag:
            for (topic, msg, ts) in bag.read_messages(topics=str(pose_topic)):
                if msg_type == "geometry_msgs/PoseStamped":
                    orientation_list = [msg.pose.orientation.x, msg.pose.orientation.y, msg.pose.orientation.z, msg.pose.orientation.w]
                    (roll, pitch, yaw) = euler_from_quaternion (orientation_list)
                    f.write('%.12f, %.12f, %.12f, %.12f, %.12f, %.12f, %.12f\n' %
                            (msg.header.stamp.to_sec(),
                             msg.pose.position.x, 
                             msg.pose.position.y,
                             msg.pose.position.z,
                             roll,
                             pitch,
                             yaw
                             )
                            )
                else:
                    assert False, "Unknown message type"
                n += 1
        print('wrote ' + str(n) + ' husky pose messages to the file: ' + out_filename)


    if __name__ == '__main__':
        parser = argparse.ArgumentParser(description='''
        Extracts IMU messages from bagfile.
        ''')
        parser.add_argument('bag', help='Bagfile')
        parser.add_argument('topic', help='Topic')
        parser.add_argument('--msg_type', default='geometry_msgs/PoseStamped',
                            help='message type')
        parser.add_argument('--output', default='husky_poses.csv',
                            help='output filename')
        args = parser.parse_args()

        out_dir = os.path.dirname(os.path.abspath(args.bag))
        out_fn = os.path.join(out_dir, args.output)

        print('Extract pose from bag '+args.bag+' in topic ' + args.topic)
        print('Saving to file '+out_fn)
        extract(args.bag, args.topic, args.msg_type, out_fn)

### change_to_ang.py

In [ ]:
def change_to_ang():

    filename = 'waypoint.txt'
    outfile = 'waypoint.csv'

    f = open(outfile, 'w')

    csv_data=pd.read_csv(open(filename))
    n = 0
    f.write('x,y,z,yaw,roll,pitch\n')
    for i in range(len(csv_data.x)):
        tempdata=[csv_data.q1[i],csv_data.q2[i],csv_data.q3[i],csv_data.q4[i]]
        (roll, pitch, yaw) = euler_from_quaternion (tempdata)
        f.write('%.12f, %.12f, %.12f, %.12f, %.12f, %.12f\n' %
                            (
                             csv_data.x[i],
                             csv_data.y[i],
                             csv_data.z[i],
                             yaw,
                             roll,
                             pitch
                             )
                )
        n += 1
    print('change ' + str(n) + ' Quaternions to angle: ' + outfile)

    f.close()

### date_caculate.py 

In [ ]:
def date_caculate():

    filename = 'echo_mb.txt'
    outfile = 'movebase_time.csv'

    with open(filename, 'r') as f:
        rows = f.readlines()

    result = [(rows[3+i],rows[4+i],rows[12+i]) for i in range(0,len(rows),17)]   

    def find_num(x):
        return re.findall('[\d.]+',x)[0]  
    data = [list(map(find_num,(sec,nsec,status))) for sec,nsec,status in result]
    # print(data)

    data = [(sec + '.' + nsec,status) for sec,nsec,status in data]
    df = pd.DataFrame(data, columns = ['time','status'])

    df.time = df.time.apply(float)
    df.status = df.status.apply(int)

    df.to_csv(outfile, index=False)

### get_erro.py

In [ ]:
def get_erro():
        
    filename = 'movebase_time.csv'
    filename_husky = 'husky_poses.csv'
    point_file = 'waypoint.csv'
    outfile = 'date_result.csv'

    df = pd.read_csv(filename)
    point = pd.read_csv(point_file)
    husky = pd.read_csv(filename_husky)

    df = df[df.status == 3].reset_index(drop = True)   #消除状态标志不是3的数据

    #数据的筛选，取时间戳前后0.1秒的数据进行求和平均
    data = []
    for i in range(len(df.time)):
        pose_temp = husky[(husky.timestamp > (df.time[i] - 0.1)) & (husky.timestamp < (df.time[i] + 0.1))]
        entry = pose_temp.mean().values
        columns_index = pose_temp.mean().index
        data.append(entry)

    df_1 = pd.DataFrame(data, columns = columns_index) 

    #对获取的数据进行筛选，去到达目标点的第一个时刻的数据
    temp_x = df_1.x[0]
    temp_y = df_1.y[0]
    pose = []
    pose.append([df_1.timestamp[0], df_1.x[0], df_1.y[0], df_1.z[0], df_1.yaw[0], df_1.roll[0], df_1.pitch[0]])
    for i in range(len(df_1.x) - 1):
        if ((df_1.x[i] - temp_x)**2 + (df_1.y[i] - temp_y)**2)**0.5 < 1:
            continue
        else:
            temp_x = df_1.x[i]
            temp_y = df_1.y[i]
            pose.append([df_1.timestamp[i], df_1.x[i], df_1.y[i], df_1.z[i], df_1.yaw[i], df_1.roll[i], df_1.pitch[i]])
    pose
    list_1 = ['timestamp', 'x','y','z','yaw','roll','pitch']
    df_1 = pd.DataFrame(pose, columns = list_1)  #获取数据
    df_2 = df_1.drop(['timestamp'], axis = 1)  #消除时间戳的信息，与waypoint.csv文件格式一致，用于对比

    times = len(df_2)//len(point)
    re = len(df_2)%len(point)

    point_sum = point
    for i in range(times - 1):
        point_sum = point_sum.append(point).reset_index().drop(['index'], axis = 1)
    point_sum = point_sum.append(point[0:re]).reset_index().drop(['index'], axis = 1)

    erro = df_2 - point_sum
    erro['timestamp'] = df_1.timestamp
    erro = erro.reindex(columns=list_1)
    test = df_1.diff(1)

    erro['time'] = test.timestamp
    erro['dist'] = (test.x**2 + test.y**2)**0.5
    erro['vel'] = erro['dist'] / erro['time']
    erro.to_csv(outfile, index = False)

### get_picture.py

In [ ]:
 def get_picture():
        
    filename = 'date_result.csv'
    outfile = 'out_plot.csv'


    df = pd.read_csv(filename)

    df['sq_x_y'] = (df.x**2 + df.y**2)**0.5
    df_1 = df.shift(1)  #数据向后移动一行

    data = {'point':[1,2,3,4,5,6,7,8,9]}  #创建新的dataframe
    df_erro = pd.DataFrame(data)

    df_erro['loop_1'] = df_1.sq_x_y[0:9].reset_index(drop = True)
    df_erro['loop_2'] = df_1.sq_x_y[9:18].reset_index(drop = True)
    df_erro['loop_3'] = df_1.sq_x_y[18:27].reset_index(drop = True)
    df_erro['loop_4'] = df_1.sq_x_y[27:36].reset_index(drop = True)
    df_erro['loop_5'] = df_1.sq_x_y[36:].reset_index(drop = True)

    df_erro['loop_1_yaw'] = df_1.yaw[0:9].reset_index(drop = True)
    df_erro['loop_2_yaw'] = df_1.yaw[9:18].reset_index(drop = True)
    df_erro['loop_3_yaw'] = df_1.yaw[18:27].reset_index(drop = True)
    df_erro['loop_4_yaw'] = df_1.yaw[27:36].reset_index(drop = True)
    df_erro['loop_5_yaw'] = df_1.yaw[36:].reset_index(drop = True)

    df_erro.to_csv(outfile, index = False)

    df_erro = abs(df_erro) #取角度误差的绝对值

    #画散点图，点的误差
    plt.plot(df_erro.point, df_erro.loop_1, '.')
    plt.plot(df_erro.point, df_erro.loop_2, '*')
    plt.plot(df_erro.point, df_erro.loop_3, '<')
    plt.plot(df_erro.point, df_erro.loop_4, '^')
    plt.plot(df_erro.point, df_erro.loop_5, '>')

    plt.xlabel('points')
    plt.ylabel('erro/m')
    plt.legend(loc = 'best')
    plt.title('points erro for 5 loops')
    plt.savefig('./point.jpg', dpi = 900, bbox_inches = 'tight')
    plt.show()

    #画散点图，角度的误差
    plt.plot(df_erro.point, df_erro.loop_1_yaw, '.')
    plt.plot(df_erro.point, df_erro.loop_2_yaw, '*')
    plt.plot(df_erro.point, df_erro.loop_3_yaw, '>')
    plt.plot(df_erro.point, df_erro.loop_4_yaw, '^')
    plt.plot(df_erro.point, df_erro.loop_5_yaw, '<')

    plt.xlabel('points')
    plt.ylabel('erro/rad')
    plt.legend(loc = 'best')
    plt.title('angle erro for 5 loops')
    plt.savefig('./yaw.jpg', dpi = 900, bbox_inches = 'tight')
    plt.show()